In [1]:
import polars_talib as plta
import polars as pl
import numpy as np

/tmp/ipykernel_170448/1891950986.py:1: DeprecationWarning: `_get_shared_lib_location` is deprecated. It will be removed in the next breaking release. The new `register_plugin_function` function has this functionality built in. Use `from polars.plugins import register_plugin_function` to import that function. Check the user guide for the currently-recommended way to register a plugin: https://docs.pola.rs/user-guide/expressions/plugins
  import polars_talib as plta


In [2]:
class PureRichmanAgent:
    def __init__(self, order_size: float=0.01, timebar_interval_sec: int=4 * 60 * 60, atr_multiple:float = 0.5, atr_bars: int=24):
        # agentの設定
        self.setting = {
            "order_size": order_size,
            "timebar_interval_sec": timebar_interval_sec,
            "atr_multiple": atr_multiple,
            "atr_bars": atr_bars
        }

        # タイムバーをagent内で保持しておくためのデータフレーム
        self.df_timebar = pl.DataFrame(schema={
            "timestamp": pl.Int64,
            "open": pl.Float64,
            "high": pl.Float64,
            "low": pl.Float64,
            "close": pl.Float64,
            "volume": pl.Float64,
            "count": pl.UInt32, 
        })

    def on_init(self, session):
        # on_clock呼び出し間隔を設定
        session.clock_interval_sec = self.setting["timebar_interval_sec"]

    def on_clock(self, session, clock):
        # rbotから最新のタイムバーをデータフレーム(1行)として取得
        _df_timebar = session.ohlcv(self.setting["timebar_interval_sec"], １)

        if _df_timebar.shape[0] > 0:
            if _df_timebar["timestamp"][-1] == clock:
                # ウォームアップ中の呼び出しなので何もしない
                return
            else:
                # バックテスト中でかつ直前のタイムバーが生成されているので、ローカルのタイムバー保持用データフレームに追加
                self.df_timebar = self.df_timebar[-12:].vstack(_df_timebar[-1]).sort("timestamp", descending = False)
        else:
            # 直前のタイムバー期間でトレードがなく、直前のタイムバーが生成されていない
            if self.df_timebar.shape[0] > 0:
                # 直前のタイムバーのクローズ価格が取得できる場合は、価格変化なしのタイムバーを作成してローカルのデータフレームに追加
                _close = self.df_timebar["close"][-1]
                _df_timebar = pl.DataFrame({
                    "timestamp": clock,
                    "open": _close,
                    "high": _close,
                    "low": _close,
                    "close": _close,
                    "volume": 0.0,
                    "count": np.uint32(0)
                })
                self.df_timebar = self.df_timebar.vstack(_df_timebar)
            else:
                # 直前のクローズ価格が取得できないので、何もしない
                raise Exception("No past timebar data & No new timebar data")
        
        # ATRの計算
        _df_timebar_with_indicators = self.df_timebar.with_columns(
            plta.atr(pl.col("high"), pl.col("low"), pl.col("close"), self.setting["atr_bars"]).alias("atr"),
            (np.log(pl.col("close")) - np.log(pl.col("close").shift(1))).alias("lr")
        )
        _atr = _df_timebar_with_indicators["atr"][-1]
        _close = _df_timebar_with_indicators["close"][-1]

        if np.isnan(_atr) == False:
            # ATRが計算できたので、注文を出す

            # 残っている指値注文を全てキャンセル
            for _order in session.buy_orders:
                session.cancel_order(_order.order_id)
                print(f"on_clock ({clock}): cancel BUY LIMIT order: {_order}")
            for _order in session.sell_orders:
                session.cancel_order(_order.order_id)
                print(f"on_clock ({clock}): cancel SELL LIMIT order: {_order}")

            if session.position <= 0.0:
                # ショートポジションを取っているか、ポジションがないので買い指値を入れる
                _price = _close - self.setting["atr_multiple"] * _atr
                _amount = self.setting["order_size"] if session.position == 0.0 else abs(session.position)

                print(f"on_clock ({clock}): BUY LIMIT ORDER {_price}, {_amount}")
                session.limit_order("Buy", _price, _amount)
            if session.position >= 0.0:
                # ロングポジションを取っているか、ポジションがないので売り指値を入れる
                _price = _close + self.setting["atr_multiple"] * _atr
                _amount = self.setting["order_size"] if session.position == 0.0 else abs(session.position)

                print(f"on_clock ({clock}): SELL LIMIT ORDER {_price}, {_amount}")
                session.limit_order("Sell", _price, _amount)
    
    def on_update(self, session, update):
        # print(f"on_update: {update}")
        pass

In [3]:
from rbot import Bybit, BybitConfig

exchange = Bybit(production=True)
config = BybitConfig.BTCUSDT          # use BTC/USDT pair
market = exchange.open_market(config)
market.expire_unfix_data(force=True)

In [4]:
#market.download_archive(
#    ndays=365,        # specify from past days
#    force=True,    # if false, the cache data will be used.
#    verbose=True    # verbose to print download progress.
#)

In [5]:
from rbot import Runner
from rbot import NOW, DAYS

agent = PureRichmanAgent(0.01, 4 * 60 * 60, 0.5, 12)
runner = Runner()

session = runner.back_test(
                exchange=exchange,
                market=market,
                agent=agent,
                start_time=0,
                end_time=0,
                verbose=False
            )

session

new session
------- warm up end ---------


/tmp/ipykernel_170448/918465157.py:58: DeprecationWarning: `register_plugin` is deprecated. Use `polars.plugins.register_plugin_function` instead.
  plta.atr(pl.col("high"), pl.col("low"), pl.col("close"), self.setting["atr_bars"]).alias("atr"),
/home/wannabebotter/github/rustybot_exercise/venv/lib/python3.10/site-packages/polars/series/series.py:1465: RuntimeWarning: divide by zero encountered in log
  series = f(lambda out: ufunc(*args, out=out, dtype=dtype_char, **kwargs))


on_clock (1689220800000000): BUY LIMIT ORDER 30228.4125, 0.01
on_clock (1689220800000000): SELL LIMIT ORDER 30342.1875, 0.01
on_clock (1689235200000000): cancel BUY LIMIT order: {"category":"linear","symbol":"BTCUSDT","create_time":1689220800000000,"status":"New","order_id":"PureRichmanAgent-p62jmB0001","client_order_id":"PureRichmanAgent-p62jmB0001","order_side":"Buy","order_type":"Limit","order_price":"30228.41","order_size":"0.01","remain_size":"0.01","transaction_id":"","update_time":1689220800345500,"execute_price":"0.0","execute_size":"0.0","quote_vol":"0.0","commission":"0.0","commission_asset":"USDT","is_maker":true,"message":"","commission_home":"0.0","commission_foreign":"0.0","home_change":"0.0","foreign_change":"0.0","free_home_change":"-302.2841","free_foreign_change":"0.0","lock_home_change":"302.2841","lock_foreign_change":"0.0","open_position":"0.0","close_position":"0.0","position":"0.0","profit":"0.0","fee":"0","total_profit":"0","log_id":1}
on_clock (1689235200000000

{"timestamp":1720569599678100,"timestamp_str": 2024-07-09T23:59:59.678100,
"orders":{"buy":{"asc":false,"list":[{"category":"linear","symbol":"BTCUSDT","create_time":1720555200000000,"status":"New","order_id":"PureRichmanAgent-p62jmB3369","client_order_id":"PureRichmanAgent-p62jmB3369","order_side":"Buy","order_type":"Limit","order_price":"57512.40","order_size":"0.01","remain_size":"0.01","transaction_id":"","update_time":1720555200124800,"execute_price":"0.0","execute_size":"0.0","quote_vol":"0.0","commission":"0.0","commission_asset":"USDT","is_maker":true,"message":"","commission_home":"0.0","commission_foreign":"0.0","home_change":"0.0","foreign_change":"0.0","free_home_change":"-575.1240","free_foreign_change":"0.0","lock_home_change":"575.1240","lock_foreign_change":"0.0","open_position":"0.0","close_position":"0.0","position":"-0.01","profit":"0.0","fee":"0","total_profit":"0","log_id":10418}]}, "sell":{"asc":true,"list":[]}}, 
"account":{"coins":[]}, "psudo_account":{"coins":[

In [6]:
log = session.log
orders = log.orders

print(f"トータルリターン (手数料込) : {orders['sum_profit'][-1]}")
print(f"トータルリターン (手数料抜き) : {orders['profit'][-1]}")

トータルリターン (手数料込) : 43.90661959999953
トータルリターン (手数料抜き) : 0.0
